In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def integrand(xdata: np.array) -> float:
    return np.sum(xdata) ** 2


# Monte Carlo Integration
def MCIntegrate(f, N: int):
    '''
    f: function to integrate -- integrand
    n: number of dimensions
    N: number of samples
    returns: integral result and integral error    
    '''
    sum = 0.0
    res = []
    relativeError = []
    for i in range(1, N+1):
        sum += f(np.random.rand(10))
        res.append(sum/i)
        relativeError.append(abs(res[-1] - 155/6))
    return res, relativeError


N = 1000000
Nlist = range(1, N+1)
res, relativeError = MCIntegrate(integrand, N)

fig = plt.figure()
plt.title("Integral Result")
plt.xlabel('N')
plt.ylabel('MC integral')
plt.ylim(25.5, 26.5)
plt.plot(Nlist, res)

fig = plt.figure()
plt.title("Integral relative error")
plt.plot([1/np.sqrt(N) for N in Nlist][10000:-1], relativeError[10000:-1])
plt.xlabel('$1/\sqrt{N}$')
plt.ylabel('relative error')

plt.show()


In [ ]:
import numpy as np
# from vpython import *


# scene = canvas(title="Strings and masses configuration", width = 500, height = 500)
# tempe = curve(x=range(0, 500), color=color.black)

n = 9  # number of equations
eps = 1e-6  # tolerance

deriv = np.zeros((n, n), float)     # derivative of f[i] with respect to x[j]
f = np.zeros((n), float)
x = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 1.0])


def plotconfig():
    # for obj in scene.objects:  # to erase the previous configuration
    #     obj.visible = 0

    L1 = 3.0
    L2 = 4.0
    L3 = 4.0

    xa = L1 * x[3]          # L1*cos(th1)
    ya = L1 * x[0]          # L1*sin(th1)
    xb = xa + L2 * x[4]     # L1*cos(th1) + L2*cos(th2)
    yb = ya + L2 * x[1]     # L1*sin(th1) + L2*sen(th2)
    xc = xb + L3 * x[5]     # L1*cos(th1) + L2*cos(th2) + L3*cos(th3)
    yc = yb - L3 * x[2]     # L1*sin(th1) + L2*sen(th2) - L3*sin(th3)

    mx = 100.0              # for linear coordinate transformation
    bx = -500.0             # from     0 <=      x       <= 10
    my = -100.0             # to    -500 <=   x_window   <= 500
    by = 400.0              # same transformation for y

    xap = mx * xa + bx      # to keep aspect ratio
    yap = my * ya + by

    xbp = mx * xb + bx
    ybp = my * yb + by

    xcp = mx * xc + bx
    ycp = my*yc + by
    x0 = mx*0 + bx
    y0 = my*0 + by

    # ball1 = sphere(pos=vector(xap, yap, 0), color=color.cyan, radius=15)
    # ball2 = sphere(pos=vector(xbp, ybp, 0), color=color.cyan, radius=25)
    # line1 = curve(pos=[(x0, y0, 0), (xap, yap, 0)], color=color.yellow, radius=4)
    # line2 = curve(pos=[(xap, yap, 0), (xbp, ybp, 0)], color=color.yellow, radius=4)
    # line3 = curve(pos=[(xbp, ybp, 0), (xcp, ycp, 0)], color=color.yellow, radius=4)
    # topline = curve(pos=[(x0, y0, 0), (xcp, ycp, 0)], color=color.red, radius=4)


def F(x, f):
    '''
    Define F function
    '''
    f[0] = 3*x[3] + 4*x[4] + 4*x[5] - 8.0
    f[1] = 3*x[0] + 4*x[1] - 4*x[2]
    f[2] = x[6]*x[0] - x[7]*x[1] - 10.0
    f[3] = x[6]*x[3] - x[7]*x[4]
    f[4] = x[7]*x[1] + x[8]*x[2] - 20.0
    f[5] = x[7]*x[4] - x[8]*x[5]
    f[6] = np.power(x[0], 2) + np.power(x[3], 2) - 1.0
    f[7] = np.power(x[1], 2) + np.power(x[4], 2) - 1.0
    f[8] = np.power(x[2], 2) + np.power(x[5], 2) - 1.0
    return f


def dFi_dXj(x, deriv, n):
    '''
    Define derivative function
    '''
    h = 1e-4  # step size
    for j in range(n):
        temp = x[j]             # store the original value of x[j]
        x[j] = x[j] + h/2.0
        F(x, f)                 # compute f(x+h/2)
        for i in range(n):
            deriv[i, j] = f[i]  # store f[i] when x[j] is slightly perturbed
        x[j] = temp             # restore the original value of x[j]

    for j in range(n):
        temp = x[j]                                 # store the original value of x[j]
        x[j] = x[j] - h/2.0
        F(x, f)                                     # compute f(x-h/2)
        for i in range(n):
            deriv[i, j] = (deriv[i, j]-f[i]) / h    # compute (f(x+h/2)-f(x-h/2))/h, so deriv[i,j] is the derivative of f[i] with respect to x[j]
        x[j] = temp                                 # restore the original value of x[j]


# store the derivative of f[i] with respect to x[j] in deriv[i,j]


for it in range(1, 100):
    F(x, f)  # compute all the f(x)
    # compute all the derivatives
    dFi_dXj(x, deriv, n)

    B = - f.reshape(n, 1)               # B = - f (a colume vector)
    sol = np.linalg.solve(deriv, B)     # solve
    dx = sol[:, 0]                      # take the first column of matrix sol

    for i in range(n):
        x[i] = x[i] + dx[i]
    # plotconfig()
    errX = errF = errXi = 0.0

    for i in range(n):
        if (x[i] != 0.):
            errXi = abs(dx[i]/x[i])
        else:
            errXi = abs(dx[i])
        if (errXi > errX):
            errX = errXi
        if (abs(f[i]) > errF):
            errF = abs(f[i])
        if ((errX <= eps) and (errF <= eps)):
            break


print('Number of iterations = ', it)
print('Solution:')

for i in range(n):
    print('x[', i, '] =  ', x[i])
